# Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

No test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\input\test 
No additional test data found at D:\vfomin\MyExamples\python\planet_amazon_rainforest\input\test 


Using TensorFlow backend.


### Load predictions on trainval dataset

In [4]:
from data_utils import unique_tags, get_label

target_tags = ['target_' + t for t in unique_tags]
val_predictions_filepath = os.path.join(GENERATED_DATA, "val_predictions.csv")
if not os.path.exists(val_predictions_filepath):
    
    val_predictions_csv = glob(os.path.join(OUTPUT_PATH, "val_predictions_*.csv"))
    df = pd.read_csv(val_predictions_csv[0]).dropna()
    for filepath in val_predictions_csv[1:]:
        df = pd.concat([df, pd.read_csv(filepath).dropna()])
    df.reset_index(inplace=True)   
    df.drop('index', axis=1, inplace=True)
    df['image_id'] = df['image_name'].apply(lambda x: int(x[len('test_'):]))    
    for t in target_tags:
        df[t] = ''
    def fill_target_tags(row):
        image_id = row[0]
        labels = get_label(image_id, "Train_jpg")
        row[1:] = labels    
        return row
    cols = ['image_id', ] + target_tags
    df[cols] = df[cols].apply(fill_target_tags, axis=1)
    
    df.to_csv(val_predictions_filepath, index=False)
    val_predictions_df = df
    df = None    
else:
    val_predictions_df = pd.read_csv(val_predictions_filepath)

In [5]:
print(len(val_predictions_df))
val_predictions_df.head()

80640


,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,test_628,0.071995,0.019810,0.018135,0.020460,0.015155,0.950393,0.018868,0.034112,0.073635,...,0,0,0,0,0,1,0,0,0,0
1,test_8842,0.070470,0.024685,0.020838,0.020732,0.016295,0.946937,0.013844,0.024798,0.065312,...,0,0,0,0,0,1,0,0,0,0
2,test_11877,0.760127,0.092261,0.118277,0.099680,0.101711,0.741155,0.107806,0.115136,0.207027,...,0,0,1,0,0,1,1,0,0,0
3,test_2413,0.229211,0.058638,0.096851,0.112940,0.040866,0.833260,0.051784,0.093998,0.156023,...,0,0,0,0,0,1,0,0,0,1
4,test_28838,0.071773,0.017712,0.019611,0.027369,0.016949,0.962932,0.013631,0.027329,0.066544,...,0,0,0,0,0,1,0,0,0,0


## Simple probability mean 

In [58]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*.csv"))
predictions_csv

['D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_0_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_10_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_11_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_12_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_13_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_14_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_1_SqueezeNet21_all_classes_fold=4_seed=

In [59]:
predictions = []
for csv_filepath in predictions_csv:
    predictions.append(pd.read_csv(csv_filepath))

In [60]:
def compute_mean(predictions):
    df = predictions[0]
    for p in predictions[1:]:
        df = pd.concat([df, p], axis=0)
    df = df.apply(pd.to_numeric, errors='ignore')
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

def compute_median(predictions):
    df = predictions[0]
    for p in predictions[1:]:
        df = pd.concat([df, p], axis=0)
    df = df.apply(pd.to_numeric, errors='ignore')
    gb = df.groupby('image_name')
    df2 = gb.agg(np.median).reset_index()
    return df2

In [61]:
mean_predictions_df = compute_mean(predictions)
median_predictions_df = compute_median(predictions)

In [64]:
mean_predictions_df.head(10)

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,file_0,0.075577,0.022042,0.028876,0.018089,0.032277,0.903471,0.013046,0.059221,0.093378,0.027963,0.020402,0.083580,0.967447,0.053680,0.035005,0.046575,0.088839
1,file_1,0.035373,0.011506,0.014149,0.007825,0.020209,0.938606,0.004963,0.037624,0.064372,0.013600,0.006733,0.056471,0.984725,0.029025,0.024612,0.022560,0.068315
2,file_10,0.449505,0.034339,0.059886,0.027366,0.057424,0.431175,0.063071,0.088553,0.169601,0.047239,0.087217,0.583021,0.913226,0.166780,0.118563,0.112523,0.232856
3,file_100,0.078833,0.020902,0.019649,0.012992,0.025159,0.939492,0.007644,0.047085,0.070833,0.030681,0.010401,0.059309,0.973639,0.079619,0.026858,0.032322,0.110833
4,file_1000,0.281819,0.043391,0.059090,0.035357,0.050177,0.832329,0.022377,0.087638,0.141859,0.083981,0.046980,0.133932,0.937326,0.180613,0.047837,0.055389,0.167381
5,file_10000,0.085706,0.022563,0.036282,0.016231,0.039441,0.895553,0.016398,0.067165,0.095148,0.024324,0.019406,0.108142,0.967313,0.064281,0.041502,0.048651,0.095327
6,file_10001,0.474047,0.058437,0.083657,0.056930,0.072907,0.624511,0.067812,0.115108,0.236861,0.126087,0.116586,0.320276,0.879582,0.260484,0.096881,0.121661,0.223934
7,file_10002,0.434395,0.039969,0.051772,0.042413,0.061335,0.528714,0.101766,0.099048,0.091020,0.115036,0.158139,0.175994,0.753081,0.300542,0.060542,0.077807,0.350775
8,file_10003,0.535194,0.073768,0.099317,0.087018,0.120431,0.485638,0.173114,0.170229,0.191334,0.174677,0.159231,0.393673,0.762687,0.374173,0.147259,0.167891,0.278456
9,file_10004,0.012178,0.005758,0.006582,0.004428,0.010561,0.974356,0.001948,0.023095,0.031437,0.007089,0.003283,0.025492,0.990363,0.014006,0.013161,0.014274,0.050892


In [63]:
median_predictions_df.head(10)

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,file_0,0.066559,0.019772,0.025201,0.017108,0.030118,0.905337,0.008332,0.059949,0.086465,0.025206,0.010023,0.067069,0.980167,0.038068,0.033692,0.037255,0.096182
1,file_1,0.030922,0.011028,0.011952,0.006366,0.018802,0.941030,0.003811,0.037193,0.067637,0.011012,0.004476,0.045566,0.988867,0.025853,0.025179,0.018345,0.070694
2,file_10,0.455552,0.023405,0.038562,0.016684,0.053297,0.444875,0.050537,0.085154,0.165315,0.031064,0.080647,0.615260,0.920209,0.156576,0.119304,0.104063,0.228729
3,file_100,0.088465,0.013756,0.017055,0.009134,0.018948,0.951660,0.006462,0.045767,0.061058,0.029359,0.003884,0.035721,0.977905,0.076777,0.019094,0.024870,0.113730
4,file_1000,0.288046,0.048286,0.059567,0.034974,0.046181,0.834069,0.014596,0.076847,0.134535,0.080764,0.034440,0.095145,0.939578,0.174093,0.042471,0.050763,0.168095
5,file_10000,0.067788,0.018756,0.038599,0.011527,0.040557,0.894577,0.010438,0.065776,0.090611,0.022719,0.013459,0.080753,0.981106,0.064814,0.041680,0.040983,0.101084
6,file_10001,0.459735,0.051752,0.082655,0.057057,0.068927,0.638720,0.070651,0.109004,0.234202,0.122661,0.116257,0.304150,0.877204,0.251166,0.100449,0.126123,0.246902
7,file_10002,0.420465,0.031212,0.048036,0.034196,0.061149,0.483584,0.095205,0.096038,0.077589,0.096271,0.163144,0.175591,0.747103,0.271806,0.061803,0.070643,0.361809
8,file_10003,0.535729,0.066386,0.089350,0.086664,0.118507,0.488478,0.160340,0.169882,0.197034,0.184991,0.162636,0.367418,0.764343,0.381978,0.146781,0.169777,0.262957
9,file_10004,0.008908,0.004451,0.006053,0.002124,0.011182,0.972631,0.001157,0.024345,0.029851,0.006934,0.000813,0.015355,0.996298,0.014409,0.013349,0.009936,0.054958


In [11]:
thresholds = {
    'agriculture': 0.35,
    'artisinal_mine': 0.35,
    'bare_ground': 0.35,
    'blooming': 0.35,
    'blow_down': 0.1,
    'clear': 0.5,
    'cloudy': 0.35,
    'conventional_mine': 0.35,
    'cultivation': 0.35,
    'habitation': 0.35,
    'haze': 0.35,
    'partly_cloudy': 0.35,
    'primary': 0.5,
    'road': 0.35,
    'selective_logging': 0.35,
    'slash_burn': 0.1,
    'water': 0.35
}

#### Search for better thresholds

In [15]:
from data_utils import unique_tags, get_label, TRAIN_ENC_CL_CSV
from metrics import score

In [13]:
y_true = val_predictions_df[target_tags].values

In [17]:
best_thresholds = {}


def search_best_threshold(y_true, y_preds):
    _thr = 0.5    
    _bs = score(y_true, y_preds > _thr)
    
    thrs = np.arange(0.0, 1.0, 0.01)
    for thr in thrs:
        s = score(y_true, y_preds > thr)
        if s > _bs:
            _bs = s
            _thr = thr      
    return _thr, _bs


for tag_index, tag in enumerate(unique_tags):
    best_thresholds[tag], best_score = search_best_threshold(y_true[:, tag_index], val_predictions_df[unique_tags].values[:, tag_index])
    print("%s | best threshold : %f with score: %f" % (tag, best_thresholds[tag], best_score))


agriculture | best threshold : 0.150000 with score: 0.775666
artisinal_mine | best threshold : 0.090000 with score: 0.106048
bare_ground | best threshold : 0.080000 with score: 0.197069
blooming | best threshold : 0.000000 with score: 0.040729
blow_down | best threshold : 0.030000 with score: 0.013234
clear | best threshold : 0.390000 with score: 0.943377
cloudy | best threshold : 0.210000 with score: 0.742016
conventional_mine | best threshold : 0.070000 with score: 0.019324
cultivation | best threshold : 0.110000 with score: 0.460947
habitation | best threshold : 0.140000 with score: 0.568400
haze | best threshold : 0.220000 with score: 0.654422
partly_cloudy | best threshold : 0.170000 with score: 0.623586
primary | best threshold : 0.360000 with score: 0.985245
road | best threshold : 0.190000 with score: 0.706468
selective_logging | best threshold : 0.030000 with score: 0.042481
slash_burn | best threshold : 0.080000 with score: 0.032576
water | best threshold : 0.110000 with scor

In [65]:
create_submission(median_predictions_df, info="squeezenet21_median_5_folds", thresholds=best_thresholds)

Mean F2 score : 0.82186

GMean F2 score : 0.68523

Median F2 score : 0.68836


## Train xgboost trees to make better predictions

In [357]:
val_predictions_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,...,target_conventional_mine,target_cultivation,target_habitation,target_haze,target_partly_cloudy,target_primary,target_road,target_selective_logging,target_slash_burn,target_water
0,test_628,0.071995,0.019810,0.018135,0.020460,0.015155,0.950393,0.018868,0.034112,0.073635,...,0,0,0,0,0,1,0,0,0,0
1,test_8842,0.070470,0.024685,0.020838,0.020732,0.016295,0.946937,0.013844,0.024798,0.065312,...,0,0,0,0,0,1,0,0,0,0
2,test_11877,0.760127,0.092261,0.118277,0.099680,0.101711,0.741155,0.107806,0.115136,0.207027,...,0,0,1,0,0,1,1,0,0,0
3,test_2413,0.229211,0.058638,0.096851,0.112940,0.040866,0.833260,0.051784,0.093998,0.156023,...,0,0,0,0,0,1,0,0,0,1
4,test_28838,0.071773,0.017712,0.019611,0.027369,0.016949,0.962932,0.013631,0.027329,0.066544,...,0,0,0,0,0,1,0,0,0,0


In [6]:
import xgboost as xgb

d:\vfomin\myexamples\venv3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### A simple try of xgb

In [7]:
from sklearn.model_selection import KFold

In [8]:
n_folds = 5

In [10]:
kf = KFold(n_splits=n_folds)

trainval_x = val_predictions_df[unique_tags].values
trainval_y = val_predictions_df[target_tags].values
    
for train_index, test_index in kf.split(trainval_x):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y[train_index], trainval_y[test_index]
    
#     print(train_x.shape, train_y.shape)
#     print(train_x[:5, :], train_y[:5])
#     print(val_x[:5, :], val_y[:5])
    break

(64512, 17) (64512, 17)
[[ 0.09093951  0.02054392  0.02418912  0.01544789  0.03247925  0.92379266
   0.00828369  0.04989937  0.10721629  0.0278988   0.00523409  0.08840037
   0.97719461  0.06283034  0.03732468  0.03562859  0.09488261]
 [ 0.05455842  0.01311656  0.02213255  0.01007489  0.03110504  0.91531706
   0.00602263  0.0469023   0.12840895  0.02287883  0.00794587  0.05563896
   0.98842609  0.03919864  0.03108688  0.02699875  0.06988449]
 [ 0.01405455  0.00637132  0.00833274  0.00247492  0.01473563  0.95992196
   0.00172729  0.03102464  0.03605482  0.00775872  0.00121341  0.03003966
   0.99548018  0.01660914  0.023029    0.01427789  0.05433382]
 [ 0.45359641  0.01025823  0.04004937  0.00729494  0.03303159  0.48314595
   0.02731058  0.06863047  0.13719328  0.02727759  0.02906365  0.58874065
   0.92141306  0.10537007  0.12347737  0.07908679  0.10379606]
 [ 0.1899071   0.01707581  0.02652191  0.01242715  0.06425259  0.72012264
   0.03139025  0.06736487  0.09192603  0.04307153  0.02066

In [18]:
params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "error",
    "learning_rate": 0.01,
    "tree_method": 'exact',
    "n_estimators": 150,
    "max_depth": 3,
#     "subsample": subsample,
#     "colsample_bytree": colsample_bytree,
    "silent": False,    
}
num_boost_round = 2500
early_stopping_rounds = 100

In [19]:
tag_index = 2

In [21]:
sumpw = val_predictions_df[target_tags[tag_index]].sum()
sumnw = len(val_predictions_df[target_tags[tag_index]]) - sumpw    
scale_pos_weight = sumnw * 1.0 / sumpw

params['scale_pos_weight'] = scale_pos_weight

In [22]:
dtrain = xgb.DMatrix(train_x, train_y[:, tag_index])
dval = xgb.DMatrix(val_x, val_y[:, tag_index])

In [23]:
watchlist = [(dtrain, 'train'), (dval, 'eval')]

In [24]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

[0]	train-error:0.261858	eval-error:0.260665
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 100 rounds.
[1]	train-error:0.261858	eval-error:0.260665
[2]	train-error:0.261858	eval-error:0.260665
[3]	train-error:0.261858	eval-error:0.260665
[4]	train-error:0.261858	eval-error:0.260665
[5]	train-error:0.261858	eval-error:0.260665
[6]	train-error:0.261858	eval-error:0.260665
[7]	train-error:0.25634	eval-error:0.258247
[8]	train-error:0.261858	eval-error:0.260665
[9]	train-error:0.25634	eval-error:0.258247
[10]	train-error:0.25634	eval-error:0.258247
[11]	train-error:0.25634	eval-error:0.258247
[12]	train-error:0.25634	eval-error:0.258247
[13]	train-error:0.25634	eval-error:0.258247
[14]	train-error:0.25634	eval-error:0.258247
[15]	train-error:0.25634	eval-error:0.258247
[16]	train-error:0.25634	eval-error:0.258247
[17]	train-error:0.25634	eval-error:0.258247
[18]	train-error:0.25634	eval-error:0.258247
[

In [25]:
y_preds_ = gbm.predict(xgb.DMatrix(trainval_x), ntree_limit=gbm.best_iteration+1)

In [26]:
search_best_threshold(y_true[:, tag_index], y_preds_)

(0.54000000000000004, 0.25030916502542022)

In [28]:
search_best_threshold(y_true[:, tag_index], val_predictions_df[unique_tags].values[:, tag_index])

(0.080000000000000002, 0.19706884016377763)

In [30]:
thr1 = 0.540
thr2 = 0.08
st = 40
end = 80
print((y_preds_[st:end] > thr1).astype(np.uint8)) 
print((val_predictions_df[unique_tags].values[st:end, tag_index] > thr2).astype(np.uint8))
print(y_true[st:end, tag_index])

[0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0
 0 0 0]
[0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [315]:
score(y_true[:, tag_index], df[unique_tags[tag_index]].values > 0.09)

0.19568799780280144

Run CV with a random search of optimal parameters

In [68]:
import matplotlib.pylab as plt
%matplotlib inline

In [69]:
def generate_params(iter_num):
    if iter_num > 0:
        for z in range(iter_num):
            print("\n-- Iteration: {}".format(z))
            eta = np.random.uniform(0.05, 0.001)
            max_depth = np.random.randint(2, 6)
            subsample = np.random.uniform(0.5, 0.95)
            colsample_bytree = np.random.uniform(0.5, 0.95)
            yield eta, max_depth, subsample, colsample_bytree
    else:
        eta = 0.05
        max_depth = 3
        subsample = 0.8204967474962096
        colsample_bytree = 0.7089159774987868
        yield eta, max_depth, subsample, colsample_bytree

In [73]:
#eta_values = [0.05, 0.025, 0.01, 0.0075, 0.005, 0.0025, 0.001]

best_params_for_tag_index = {}
for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))    
    seed = 2017
    n_folds = 5
    dtrainval = xgb.DMatrix(trainval_x, label=trainval_y[:, tag_index], feature_names=unique_tags)

    best_params_for_tag_index[tag_index] = {
        'test-%s-mean' % params['eval_metric']: 1e10,
        'params': None,
    }
    
    sumpw = val_predictions_df[target_tags[tag_index]].sum()
    sumnw = len(val_predictions_df[target_tags[tag_index]]) - sumpw    
    scale_pos_weight = sumnw * 1.0 / sumpw
    
    iter_num = 15
#     iter_num = -1   
    gen = generate_params(iter_num)
    z = 0
    for (eta, max_depth, subsample, colsample_bytree) in gen:
        z += 1
        seed += z-1
        print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth,subsample,colsample_bytree))
        params = {
            "objective": "binary:logistic",
            "booster": "gbtree",
            "eval_metric": "error",
            "eta": eta,
            "tree_method": 'exact',
            "max_depth": max_depth,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "silent": 1,
            "seed": seed, 
            "scale_pos_weight": scale_pos_weight,
        }
        num_boost_round = 2000
        early_stopping_rounds = 100

        cvresult = xgb.cv(params, dtrain=dtrainval,
                           seed=params['seed'], 
                           num_boost_round=num_boost_round, 
                           early_stopping_rounds=early_stopping_rounds, nfold=n_folds, verbose_eval=False)
                
        min_test_logloss_mean = cvresult['test-%s-mean' % params['eval_metric']].min()
        if best_params_for_tag_index[tag_index]['test-%s-mean' % params['eval_metric']] > min_test_logloss_mean:
            best_params_for_tag_index[tag_index]['test-%s-mean' % params['eval_metric']] = min_test_logloss_mean
            best_params_for_tag_index[tag_index]['params'] = params
            print("Best cv result: ", cvresult.loc[cvresult.index[-1], :])
            print("Best params: ", params)
            
        


----------------
 Tag index: 0

-- Iteration: 0
XGBoost params. ETA: 0.007796365949721863, MAX_DEPTH: 3, SUBSAMPLE: 0.8503405560516174, COLSAMPLE_BY_TREE: 0.5563111105162462
Best cv result:  test-error-mean     0.186595
test-error-std      0.003434
train-error-mean    0.185925
train-error-std     0.001135
Name: 0, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.007796365949721863, 'tree_method': 'exact', 'max_depth': 3, 'subsample': 0.8503405560516174, 'colsample_bytree': 0.5563111105162462, 'silent': 1, 'seed': 2017, 'scale_pos_weight': 2.2953291651342407}

-- Iteration: 1
XGBoost params. ETA: 0.029327248650047478, MAX_DEPTH: 3, SUBSAMPLE: 0.6186701247268119, COLSAMPLE_BY_TREE: 0.6334399849809398

-- Iteration: 2
XGBoost params. ETA: 0.04477652224975914, MAX_DEPTH: 3, SUBSAMPLE: 0.8215052088558108, COLSAMPLE_BY_TREE: 0.940372126495143

-- Iteration: 3
XGBoost params. ETA: 0.018095551076288685, MAX_DEPTH: 5, SUBSAMPLE


-- Iteration: 2
XGBoost params. ETA: 0.04477652224975914, MAX_DEPTH: 3, SUBSAMPLE: 0.8215052088558108, COLSAMPLE_BY_TREE: 0.940372126495143

-- Iteration: 3
XGBoost params. ETA: 0.018095551076288685, MAX_DEPTH: 5, SUBSAMPLE: 0.7342080530591647, COLSAMPLE_BY_TREE: 0.8980136324261054

-- Iteration: 4
XGBoost params. ETA: 0.012063976013602831, MAX_DEPTH: 3, SUBSAMPLE: 0.8135263419352342, COLSAMPLE_BY_TREE: 0.9097441644531741

-- Iteration: 5
XGBoost params. ETA: 0.037222653554939436, MAX_DEPTH: 2, SUBSAMPLE: 0.5608766985378607, COLSAMPLE_BY_TREE: 0.5918384095441328

-- Iteration: 6
XGBoost params. ETA: 0.03201077103619071, MAX_DEPTH: 2, SUBSAMPLE: 0.9036202941683869, COLSAMPLE_BY_TREE: 0.5820844773931405

-- Iteration: 7
XGBoost params. ETA: 0.048774868242772856, MAX_DEPTH: 5, SUBSAMPLE: 0.8232135887360947, COLSAMPLE_BY_TREE: 0.9247718213959808
Best cv result:  test-error-mean     0.037376
test-error-std      0.000620
train-error-mean    0.005394
train-error-std     0.000270
Name: 1999, 

Best cv result:  test-error-mean     0.003249
test-error-std      0.000473
train-error-mean    0.000360
train-error-std     0.000133
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.018095551076288685, 'tree_method': 'exact', 'max_depth': 5, 'subsample': 0.7342080530591647, 'colsample_bytree': 0.8980136324261054, 'silent': 1, 'seed': 2023, 'scale_pos_weight': 404.22613065326635}

-- Iteration: 4
XGBoost params. ETA: 0.012063976013602831, MAX_DEPTH: 3, SUBSAMPLE: 0.8135263419352342, COLSAMPLE_BY_TREE: 0.9097441644531741

-- Iteration: 5
XGBoost params. ETA: 0.037222653554939436, MAX_DEPTH: 2, SUBSAMPLE: 0.5608766985378607, COLSAMPLE_BY_TREE: 0.5918384095441328

-- Iteration: 6
XGBoost params. ETA: 0.03201077103619071, MAX_DEPTH: 2, SUBSAMPLE: 0.9036202941683869, COLSAMPLE_BY_TREE: 0.5820844773931405

-- Iteration: 7
XGBoost params. ETA: 0.048774868242772856, MAX_DEPTH: 5, SUBSAMPLE: 0.8232135887360947, COLSAM

Best cv result:  test-error-mean     0.046156
test-error-std      0.000853
train-error-mean    0.034735
train-error-std     0.000871
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.04477652224975914, 'tree_method': 'exact', 'max_depth': 3, 'subsample': 0.8215052088558108, 'colsample_bytree': 0.940372126495143, 'silent': 1, 'seed': 2020, 'scale_pos_weight': 18.122599004031301}

-- Iteration: 3
XGBoost params. ETA: 0.018095551076288685, MAX_DEPTH: 5, SUBSAMPLE: 0.7342080530591647, COLSAMPLE_BY_TREE: 0.8980136324261054
Best cv result:  test-error-mean     0.041828
test-error-std      0.000811
train-error-mean    0.027068
train-error-std     0.000553
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.018095551076288685, 'tree_method': 'exact', 'max_depth': 5, 'subsample': 0.7342080530591647, 'colsample_bytree': 0.8980136324261054, 'sil


----------------
 Tag index: 8

-- Iteration: 0
XGBoost params. ETA: 0.014117655393849675, MAX_DEPTH: 2, SUBSAMPLE: 0.5982330859669674, COLSAMPLE_BY_TREE: 0.707072909062342
Best cv result:  test-error-mean     0.357614
test-error-std      0.004359
train-error-mean    0.352796
train-error-std     0.003021
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.014117655393849675, 'tree_method': 'exact', 'max_depth': 2, 'subsample': 0.5982330859669674, 'colsample_bytree': 0.707072909062342, 'silent': 1, 'seed': 2017, 'scale_pos_weight': 8.2033782241497377}

-- Iteration: 1
XGBoost params. ETA: 0.029327248650047478, MAX_DEPTH: 3, SUBSAMPLE: 0.6186701247268119, COLSAMPLE_BY_TREE: 0.6334399849809398
Best cv result:  test-error-mean     0.328013
test-error-std      0.003787
train-error-mean    0.306882
train-error-std     0.003078
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtr


-- Iteration: 13
XGBoost params. ETA: 0.04557168327562415, MAX_DEPTH: 5, SUBSAMPLE: 0.8351662492654948, COLSAMPLE_BY_TREE: 0.5079279690320685

-- Iteration: 14
XGBoost params. ETA: 0.014530341849265056, MAX_DEPTH: 3, SUBSAMPLE: 0.5722433993736292, COLSAMPLE_BY_TREE: 0.6660731749497361

----------------
 Tag index: 10

-- Iteration: 0
XGBoost params. ETA: 0.014117655393849675, MAX_DEPTH: 2, SUBSAMPLE: 0.5982330859669674, COLSAMPLE_BY_TREE: 0.707072909062342
Best cv result:  test-error-mean     0.112648
test-error-std      0.005813
train-error-mean    0.112193
train-error-std     0.006059
Name: 0, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.014117655393849675, 'tree_method': 'exact', 'max_depth': 2, 'subsample': 0.5982330859669674, 'colsample_bytree': 0.707072909062342, 'silent': 1, 'seed': 2017, 'scale_pos_weight': 13.936099277644008}

-- Iteration: 1
XGBoost params. ETA: 0.029327248650047478, MAX_DEPTH: 3, SUBSAMP

Best cv result:  test-error-mean     0.144444
test-error-std      0.001955
train-error-mean    0.067963
train-error-std     0.000997
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.04965712898037271, 'tree_method': 'exact', 'max_depth': 5, 'subsample': 0.766586509402208, 'colsample_bytree': 0.8393513697801558, 'silent': 1, 'seed': 2083, 'scale_pos_weight': 4.569060773480663}

-- Iteration: 12
XGBoost params. ETA: 0.026594610902121417, MAX_DEPTH: 2, SUBSAMPLE: 0.8427179414316228, COLSAMPLE_BY_TREE: 0.5482230233646993

-- Iteration: 13
XGBoost params. ETA: 0.04557168327562415, MAX_DEPTH: 5, SUBSAMPLE: 0.8351662492654948, COLSAMPLE_BY_TREE: 0.5079279690320685

-- Iteration: 14
XGBoost params. ETA: 0.014530341849265056, MAX_DEPTH: 3, SUBSAMPLE: 0.5722433993736292, COLSAMPLE_BY_TREE: 0.6660731749497361

----------------
 Tag index: 12

-- Iteration: 0
XGBoost params. ETA: 0.014117655393849675, MAX_DEPTH: 2, SUBS


-- Iteration: 7
XGBoost params. ETA: 0.048774868242772856, MAX_DEPTH: 5, SUBSAMPLE: 0.8232135887360947, COLSAMPLE_BY_TREE: 0.9247718213959808

-- Iteration: 8
XGBoost params. ETA: 0.0320923888713293, MAX_DEPTH: 5, SUBSAMPLE: 0.7440565774074098, COLSAMPLE_BY_TREE: 0.6709402091242769

-- Iteration: 9
XGBoost params. ETA: 0.03624396870150058, MAX_DEPTH: 5, SUBSAMPLE: 0.6963832426114989, COLSAMPLE_BY_TREE: 0.5029876772621785

-- Iteration: 10
XGBoost params. ETA: 0.0021708158254163806, MAX_DEPTH: 5, SUBSAMPLE: 0.5383506033533576, COLSAMPLE_BY_TREE: 0.8854712489941284

-- Iteration: 11
XGBoost params. ETA: 0.04965712898037271, MAX_DEPTH: 5, SUBSAMPLE: 0.766586509402208, COLSAMPLE_BY_TREE: 0.8393513697801558

-- Iteration: 12
XGBoost params. ETA: 0.026594610902121417, MAX_DEPTH: 2, SUBSAMPLE: 0.8427179414316228, COLSAMPLE_BY_TREE: 0.5482230233646993

-- Iteration: 13
XGBoost params. ETA: 0.04557168327562415, MAX_DEPTH: 5, SUBSAMPLE: 0.8351662492654948, COLSAMPLE_BY_TREE: 0.5079279690320685



-- Iteration: 5
XGBoost params. ETA: 0.037222653554939436, MAX_DEPTH: 2, SUBSAMPLE: 0.5608766985378607, COLSAMPLE_BY_TREE: 0.5918384095441328

-- Iteration: 6
XGBoost params. ETA: 0.03201077103619071, MAX_DEPTH: 2, SUBSAMPLE: 0.9036202941683869, COLSAMPLE_BY_TREE: 0.5820844773931405

-- Iteration: 7
XGBoost params. ETA: 0.048774868242772856, MAX_DEPTH: 5, SUBSAMPLE: 0.8232135887360947, COLSAMPLE_BY_TREE: 0.9247718213959808
Best cv result:  test-error-mean     0.004985
test-error-std      0.000401
train-error-mean    0.000000
train-error-std     0.000000
Name: 1999, dtype: float64
Best params:  {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'error', 'eta': 0.048774868242772856, 'tree_method': 'exact', 'max_depth': 5, 'subsample': 0.8232135887360947, 'colsample_bytree': 0.9247718213959808, 'silent': 1, 'seed': 2045, 'scale_pos_weight': 207.9119170984456}

-- Iteration: 8
XGBoost params. ETA: 0.0320923888713293, MAX_DEPTH: 5, SUBSAMPLE: 0.7440565774074098, COLSAMPLE

In [75]:
import pickle 
now = datetime.now()

xgb_best_params_filepath = os.path.join(GENERATED_DATA, 'xgb_best_params_%s.pkl' % str(now.strftime("%Y-%m-%d-%H-%M")))

with open(xgb_best_params_filepath, 'wb') as handle:
    pickle.dump(best_params_for_tag_index, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [76]:
best_params_for_tag_index

{0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.9097441644531741,
   'eta': 0.012063976013602831,
   'eval_metric': 'error',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'scale_pos_weight': 2.2953291651342407,
   'seed': 2027,
   'silent': 1,
   'subsample': 0.8135263419352342,
   'tree_method': 'exact'},
  'test-error-mean': 0.18432560000000001},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.9247718213959808,
   'eta': 0.048774868242772856,
   'eval_metric': 'error',
   'max_depth': 5,
   'objective': 'binary:logistic',
   'scale_pos_weight': 119.17883755588673,
   'seed': 2045,
   'silent': 1,
   'subsample': 0.8232135887360947,
   'tree_method': 'exact'},
  'test-error-mean': 0.0096606000000000018},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8393513697801558,
   'eta': 0.04965712898037271,
   'eval_metric': 'error',
   'max_depth': 5,
   'objective': 'binary:logistic',
   'scale_pos_weight': 45.478386167146972,
   'seed': 

In [122]:
_best_params_for_tag_index = {
0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38209179999999998},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.036067200000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.6334399849809398,
   'eta': 0.15055304521905347,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2019,
   'silent': 1,
   'subsample': 0.6186701247268119,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.086130599999999988},
 3: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.041870600000000001},
 4: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2017,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.015638200000000001},
 5: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.30505919999999997},
 6: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.068178600000000006},
 7: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2017,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.013706000000000001},
 8: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8579978891053311,
   'eta': 0.18574545610881862,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2044,
   'silent': 1,
   'subsample': 0.8611335020320647,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.29456899999999997},
 9: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.20715659999999997},
 10: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7089159774987868,
   'eta': 0.20210354378354375,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.8204967474962096,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.1200102},
 11: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.29197859999999998},
 12: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.1244864},
 13: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5045164695165618,
   'eta': 0.1428101678052025,
   'eval_metric': 'logloss',
   'max_depth': 3,
   'objective': 'binary:logistic',
   'seed': 2016,
   'silent': 1,
   'subsample': 0.7215305289307261,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.3228318},
 14: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.8398837534294123,
   'eta': 0.14522170765829945,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2022,
   'silent': 1,
   'subsample': 0.8734804475952236,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.041907600000000003},
 15: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.586741114680066,
   'eta': 0.08503519422605446,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2052,
   'silent': 1,
   'subsample': 0.9395104239451562,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.027125},
 16: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38378999999999996}
}

{0: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.38209179999999998},
 1: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.5813435352146873,
   'eta': 0.12653607181296217,
   'eval_metric': 'logloss',
   'max_depth': 4,
   'objective': 'binary:logistic',
   'seed': 2037,
   'silent': 1,
   'subsample': 0.537796332804125,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.036067200000000001},
 2: {'params': {'booster': 'gbtree',
   'colsample_bytree': 0.6334399849809398,
   'eta': 0.15055304521905347,
   'eval_metric': 'logloss',
   'max_depth': 2,
   'objective': 'binary:logistic',
   'seed': 2019,
   'silent': 1,
   'subsample': 0.6186701247268119,
   'tree_method': 'exact'},
  'test_logloss_mean': 0.0861305999999

Train 17 binary classifiers

In [77]:
from sklearn.model_selection import train_test_split

for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    train_x, val_x, train_y, val_y = train_test_split(trainval_x, trainval_y, train_size=0.85)
    dtrain = xgb.DMatrix(train_x, train_y[:, tag_index])
    dval = xgb.DMatrix(val_x, val_y[:, tag_index])
    watchlist = [(dtrain, 'train'), (dval, 'eval')]
    params = best_params_for_tag_index[tag_index]['params']
    num_boost_round = 2500    
    early_stopping_rounds = 12

    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=False)                
    print("Best score: ", gbm.best_score)
    best_params_for_tag_index[tag_index]['gbm'] = gbm



----------------
 Tag index: 0
Best score:  0.188161

----------------
 Tag index: 1
Best score:  0.028604

----------------
 Tag index: 2
Best score:  0.271412

----------------
 Tag index: 3
Best score:  0.310764

----------------
 Tag index: 4
Best score:  0.013724

----------------
 Tag index: 5
Best score:  0.137649

----------------
 Tag index: 6
Best score:  0.065228

----------------
 Tag index: 7
Best score:  0.005704

----------------
 Tag index: 8
Best score:  0.365245

----------------
 Tag index: 9
Best score:  0.215608

----------------
 Tag index: 10
Best score:  0.111938

----------------
 Tag index: 11
Best score:  0.154349

----------------
 Tag index: 12
Best score:  0.133185

----------------
 Tag index: 13
Best score:  0.183284

----------------
 Tag index: 14
Best score:  0.026207

----------------
 Tag index: 15
Best score:  0.020007

----------------
 Tag index: 16
Best score:  0.37128


Compute best thresholds

In [91]:
best_thresholds = {}

for tag_index, tag in enumerate(unique_tags):
        
    dmat = xgb.DMatrix(trainval_x)
    gbm = best_params_for_tag_index[tag_index]['gbm']
    y_preds_ = gbm.predict(dmat, ntree_limit=gbm.best_iteration+1)        
    
    best_thresholds[tag], best_score = search_best_threshold(y_true[:, tag_index], y_preds_)
    print("%s | best threshold : %f with score: %f" % (tag, best_thresholds[tag], best_score))


agriculture | best threshold : 0.490000 with score: 0.793743
artisinal_mine | best threshold : 0.880000 with score: 0.886514
bare_ground | best threshold : 0.530000 with score: 0.265144
blooming | best threshold : 0.540000 with score: 0.128538
blow_down | best threshold : 0.880000 with score: 0.889571
clear | best threshold : 0.220000 with score: 0.950538
cloudy | best threshold : 0.520000 with score: 0.789885
conventional_mine | best threshold : 0.940000 with score: 0.886386
cultivation | best threshold : 0.500000 with score: 0.487912
habitation | best threshold : 0.520000 with score: 0.609139
haze | best threshold : 0.630000 with score: 0.705485
partly_cloudy | best threshold : 0.490000 with score: 0.640992
primary | best threshold : 0.030000 with score: 0.986853
road | best threshold : 0.490000 with score: 0.711033
selective_logging | best threshold : 0.860000 with score: 0.859974
slash_burn | best threshold : 0.880000 with score: 0.845824
water | best threshold : 0.490000 with scor

Boost test predictions

In [78]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*.csv"))

prediction_df = pd.read_csv(predictions_csv[0]).dropna()
for filepath in predictions_csv[1:]:
    prediction_df = pd.concat([prediction_df, pd.read_csv(filepath).dropna()])
prediction_df.reset_index(inplace=True)   
prediction_df.drop('index', axis=1, inplace=True)

In [79]:
prediction_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_4081,0.165832,0.033780,0.050322,0.034996,0.022693,0.684592,0.030134,0.049835,0.124013,0.024368,0.153870,0.187068,0.913896,0.073058,0.028319,0.094160,0.077560
1,test_33711,0.222213,0.045113,0.053467,0.047543,0.028283,0.852831,0.057267,0.082518,0.128163,0.031860,0.078647,0.261587,0.878942,0.137407,0.036402,0.088846,0.123938
2,test_40401,0.044391,0.012558,0.013353,0.018617,0.010823,0.966936,0.013935,0.028719,0.042504,0.003582,0.015972,0.128013,0.954551,0.022618,0.015559,0.059408,0.016636
3,test_6775,0.035926,0.015325,0.013160,0.017864,0.007701,0.974545,0.008489,0.021956,0.041470,0.002838,0.017778,0.095539,0.953173,0.016986,0.015521,0.045517,0.011710
4,test_21510,0.081408,0.029876,0.029234,0.025516,0.021830,0.944467,0.022296,0.039747,0.060446,0.007459,0.030976,0.164240,0.937648,0.046039,0.025199,0.070819,0.032278


In [80]:
y_preds_init = prediction_df[unique_tags].values
y_preds = np.zeros_like(y_preds_init)

for tag_index in range(len(unique_tags)):
    
    print("\n----------------\n Tag index: {}".format(tag_index))
    dtest = xgb.DMatrix(y_preds_init)
    gbm = best_params_for_tag_index[tag_index]['gbm']
    y_preds_ = gbm.predict(dtest, ntree_limit=gbm.best_iteration+1)        
    y_preds[:, tag_index] = y_preds_



----------------
 Tag index: 0

----------------
 Tag index: 1

----------------
 Tag index: 2

----------------
 Tag index: 3

----------------
 Tag index: 4

----------------
 Tag index: 5

----------------
 Tag index: 6

----------------
 Tag index: 7

----------------
 Tag index: 8

----------------
 Tag index: 9

----------------
 Tag index: 10

----------------
 Tag index: 11

----------------
 Tag index: 12

----------------
 Tag index: 13

----------------
 Tag index: 14

----------------
 Tag index: 15

----------------
 Tag index: 16


In [81]:
df = prediction_df.copy()
df[unique_tags] = y_preds

In [82]:
df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_4081,0.494638,0.000196,0.439053,0.429218,0.003243,0.458528,0.454417,0.000019,0.499458,0.414329,0.676011,0.499976,0.884679,0.490066,0.000491,0.000421,0.507677
1,test_33711,0.496679,0.008407,0.458891,0.539871,0.162888,0.523254,0.453436,0.002197,0.506620,0.478744,0.298366,0.499976,0.846855,0.493409,0.157152,0.052204,0.507677
2,test_40401,0.473674,0.000013,0.430307,0.521987,0.006081,0.797514,0.451714,0.000023,0.496572,0.418557,0.133207,0.482591,0.988855,0.473485,0.047266,0.000583,0.478602
3,test_6775,0.473674,0.000023,0.430307,0.521987,0.009828,0.800325,0.451714,0.000096,0.477074,0.416470,0.153449,0.478232,0.989062,0.473485,0.005638,0.000724,0.478602
4,test_21510,0.479934,0.000085,0.444572,0.521987,0.010004,0.679326,0.451714,0.000005,0.494629,0.422213,0.163609,0.482591,0.984667,0.474874,0.290488,0.063556,0.487551


In [83]:
def compute_mean(df):
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [84]:
mean_df = compute_mean(df)

In [85]:
len(mean_df), len(df)

(61191, 917865)

In [94]:
create_submission(mean_df, info="squeezenet21_blended3_5_folds", thresholds=best_thresholds)